This notebook combines the cellprofiler extracted morphology features and the scDINO extracted morphology features into one feature space. Downstream notebooks will normalize the data and perform feature selection.

In [1]:
import pathlib

import numpy as np
import pandas as pd

In [2]:
# define data paths for import
# annotated features from cellprofiler including all time points
cellprofiler_annotated_features_path = pathlib.Path(
    "../../5.process_CP_features/data/3.combined_data/profiles/combined_data.parquet"
).resolve(strict=True)

# scDINO features from the scDINO analysis including all time points
scdino_features = pathlib.Path(
    "../../6.scDINO_analysis/1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated.parquet"
).resolve(strict=True)

# set the output path
output_path = pathlib.Path(
    "../data/CP_scDINO_features/combined_CP_scDINO_data.parquet"
).resolve()

# make the parent directory
output_path.parent.mkdir(parents=True, exist_ok=True)

In [3]:
# load in the data
cellprofiler_data = pd.read_parquet(cellprofiler_annotated_features_path)
scdino_data = pd.read_parquet(scdino_features)

print(f"cellprofiler data shape: {cellprofiler_data.shape}")
print(f"scDINO data shape: {scdino_data.shape}")

cellprofiler data shape: (168577, 2332)
scDINO data shape: (145489, 1545)


In [4]:
# append either CP or scDINO to the column names
for col in cellprofiler_data.columns:
    # ensure Metadata is not in the column name
    if not "Metadata" in col:
        cellprofiler_data.rename(columns={col: f"{col}_CP"}, inplace=True)
for col in scdino_data.columns:
    # ensure Metadata is not in the column name
    if not "Metadata" in col:
        scdino_data.rename(columns={col: f"{col}_scDINO"}, inplace=True)

In [5]:
# make the Metadata Columns objects
# these are the columns that are common between the two datasets
cellprofiler_metadata_columns = [
    "Metadata_Well",
    "Metadata_FOV",
    "Metadata_Time",
    "Metadata_ImageNumber",
    "Metadata_Nuclei_Number_Object_Number",
    "Metadata_compound",
    "Metadata_dose",
    "Metadata_control",
]

In [6]:
# sort the data by Well, FOV, Time, ImageNumber, Nuclei_Number_Object_Number
cellprofiler_data = cellprofiler_data.sort_values(
    by=cellprofiler_metadata_columns,
    ascending=True,
).reset_index(drop=True)

scdino_data = scdino_data.sort_values(
    by=cellprofiler_metadata_columns,
    ascending=True,
).reset_index(drop=True)

In [7]:
scdino_data.head()
# convert time to float
scdino_data["Metadata_Time"] = scdino_data["Metadata_Time"].astype(float)
scdino_data["Metadata_Time"] = scdino_data["Metadata_Time"] - 1
scdino_data.head()

,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_compound,Metadata_dose,Metadata_control,channel488-1_cls_feature_0_scDINO,...,channel_DNA_cls_feature_90_scDINO,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO
0,../0.pre-process_images/data/processed_images/...,C-02,0001,0.0,1,102.0,Staurosporine,0.0,negative,-0.012335,...,0.035068,0.000629,-0.034905,0.056939,-0.077406,0.033168,0.005295,0.015653,0.036244,-0.013741
1,../0.pre-process_images/data/processed_images/...,C-02,0001,0.0,1,106.0,Staurosporine,0.0,negative,-0.012160,...,-0.005525,-0.032507,-0.043178,0.013190,-0.073641,0.029824,0.032743,-0.017366,0.053009,-0.006046
2,../0.pre-process_images/data/processed_images/...,C-02,0001,0.0,1,122.0,Staurosporine,0.0,negative,0.011821,...,0.015511,0.027061,0.002058,0.039717,-0.118989,0.010754,0.006840,0.048121,0.046048,0.018789
3,../0.pre-process_images/data/processed_images/...,C-02,0001,0.0,1,128.0,Staurosporine,0.0,negative,-0.008769,...,-0.009289,-0.016169,-0.035088,0.037067,-0.101932,-0.030957,0.027611,0.001648,0.090710,-0.000729
4,../0.pre-process_images/data/processed_images/...,C-02,0001,0.0,1,13.0,Staurosporine,0.0,negative,-0.000989,...,0.026018,-0.003725,0.002089,0.032937,-0.053131,0.023977,-0.018216,0.041922,0.045374,-0.033786


In [8]:
for col in cellprofiler_metadata_columns:
    if col not in cellprofiler_data.columns:
        raise ValueError(f"{col} not found in cellprofiler data.")
    cellprofiler_data[col] = cellprofiler_data[col].astype(str)
    if col not in scdino_data.columns:
        raise ValueError(f"{col} not found in scDINO data.")
    scdino_data[col] = scdino_data[col].astype(str)

In [9]:
print(f"cellprofiler data shape after sorting: {cellprofiler_data.shape}")
print(f"scDINO data shape after sorting: {scdino_data.shape}")
merged_df = pd.merge(
    cellprofiler_data,
    scdino_data,
    how="right",
    on=cellprofiler_metadata_columns,
)
print(f"merged data shape: {merged_df.shape}")
# drop duplicates
merged_df = merged_df.drop_duplicates(
    subset=cellprofiler_metadata_columns,
    keep="last",
)
print(f"merged data shape after dropping duplicates: {merged_df.shape}")

cellprofiler data shape after sorting: (168577, 2332)
scDINO data shape after sorting: (145489, 1545)
merged data shape: (149993, 3869)
merged data shape after dropping duplicates: (145489, 3869)


In [10]:
merged_df.to_parquet(output_path)
print(f"merged_df shape: {merged_df.shape}")
merged_df.head()

merged_df shape: (145489, 3869)


,Metadata_plate,Metadata_Well,Metadata_number_of_singlecells,Metadata_compound,Metadata_dose,Metadata_control,Metadata_ImageNumber,Metadata_FOV,Metadata_Time,Metadata_Cells_Number_Object_Number,...,channel_DNA_cls_feature_90_scDINO,channel_DNA_cls_feature_91_scDINO,channel_DNA_cls_feature_92_scDINO,channel_DNA_cls_feature_93_scDINO,channel_DNA_cls_feature_94_scDINO,channel_DNA_cls_feature_95_scDINO,channel_DNA_cls_feature_96_scDINO,channel_DNA_cls_feature_97_scDINO,channel_DNA_cls_feature_98_scDINO,channel_DNA_cls_feature_99_scDINO
0,1,C-02,178,Staurosporine,0.0,negative,1,0001,0.0,98.0,...,0.035068,0.000629,-0.034905,0.056939,-0.077406,0.033168,0.005295,0.015653,0.036244,-0.013741
1,1,C-02,178,Staurosporine,0.0,negative,1,0001,0.0,108.0,...,-0.005525,-0.032507,-0.043178,0.013190,-0.073641,0.029824,0.032743,-0.017366,0.053009,-0.006046
2,1,C-02,178,Staurosporine,0.0,negative,1,0001,0.0,123.0,...,0.015511,0.027061,0.002058,0.039717,-0.118989,0.010754,0.006840,0.048121,0.046048,0.018789
3,1,C-02,178,Staurosporine,0.0,negative,1,0001,0.0,127.0,...,-0.009289,-0.016169,-0.035088,0.037067,-0.101932,-0.030957,0.027611,0.001648,0.090710,-0.000729
4,1,C-02,178,Staurosporine,0.0,negative,1,0001,0.0,15.0,...,0.026018,-0.003725,0.002089,0.032937,-0.053131,0.023977,-0.018216,0.041922,0.045374,-0.033786
